In [61]:
import urllib.request, urllib.parse, urllib.error     #importing libraries urllib.request .parse and .error
from bs4 import BeautifulSoup                         #importing beautifulsoup for parsing HTML and XML sites
import ssl                                            #importing Secure Socket Layer to encrypt and decrypt the data going over the socket
import requests                                       #importing requests to send HTTP requests using python

                                                      #To ignore the SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

# Create a container with 500 movies

In [62]:
container = []
for i in range(0,5,1):                                           #0-4, each iteration will give you 100 movies
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i) #Basic url of the webpage
    page = requests.get(url, headers = headers)                  #get the url and store it in variable page
    soup = BeautifulSoup(page.content, "html.parser")            #Parse the html page and store it in variable soup
    hundred = soup.find_all('td', class_ = 'clamp-summary-wrap') #100 movies are present in the hundred list
    container.extend(hundred)                                    #extends hundred movies inside a list called container
print(len(container))                                            #Print length of the container. It should print 500 movies

500


# Define a function to get movie name

In [63]:
def get_name():                                             #Function to get the name of the movie
    name = movie.find('h3').text                            #As the movie name is written in h3 tag, we find the text in h3 tag
    return name                                             #Return the movie name    

# Define a function to get cast

In [64]:
def get_cast(detail_link):                                                     #Function to get the cast details
    detail_page = requests.get(detail_link, headers = headers)                 #get request
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")            #parse the html page
    try:                                                                       #extract the cast if summary = summary = movie_name + "'s Cast Credits"
        cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "'s Cast Credits")
        cast_line = cast_table.find_all('td', class_ = 'person')
    except:
        try:                                                                   #extract the cast if summary = summary = movie_name + "' Cast Credits"
            cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "' Cast Credits")
            cast_line = cast_table.find_all('td', class_ = 'person')
        except:
            cast_line = None                                                   #set cast_line = None is the movie does not have any cast
    casts = list()                                                             #create an empty list as casts
    if cast_line == None:                                                      #if cast_line is None, set casts as an empty list
        casts = []    
    else:
        for line in cast_line:                                                 #else, find the cast for each movie
            hyperlink = line.find('a')                                         #and append it to casts
            cast = hyperlink.get_text()
            casts.append(cast.strip()) 
        return casts                                                           #return casts

# Define a function to get Principle Cast

In [65]:
#let's then try the get cast function

def get_pri_cast(detail_link):                                          #Function to get the Principal cast details
    detail_page = requests.get(detail_link, headers = headers)          #get request
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")     #parse the html page
    try:                                                                #extract the cast if summary = movie_name + "'s Principal Cast Credits"
        pri_cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "'s Principal Cast Credits")
        pri_cast_line = pri_cast_table.find_all('td', class_ = 'person')
    except:
        try:                                                            #extract the cast if summary = movie_name + "' Principal Cast Credits"
            pri_cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "' Principal Cast Credits")
            pri_cast_line = pri_cast_table.find_all('td', class_ = 'person')
        except:
            pri_cast_line = None                                        #set cast_line = None is the movie does not have any cast
    pri_casts = list()                                                  #create an empty list as pri_casts
    if pri_cast_line == None:                                           #if pri_cast_line is None, create pri_casts as an empty list
        pri_casts = []    
    else:
        for line in pri_cast_line:                                      #else, find the cast for each movie
            hyperlink = line.find('a')                                  #and append it to casts
            pri_cast = hyperlink.get_text()
            pri_casts.append(pri_cast.strip())
        return pri_casts                                                #return pri_casts

# Define a function to get Genre

In [66]:
def get_genre_new(detail_link):                                         #function to get the genre details
    detail_page = requests.get(detail_link, headers = headers)          #get request
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")     #parse the html page
    genre_table = detail_soup.find('table', class_ = 'details')         #find table with class details
    genre_line = genre_table.find_all('tr', class_ = 'genres')          #if there is more than 1 director
    genres = []                                                         #Create an empty list as genres
    for line in genre_line:
        hyperlink = line.find('td', class_ = 'data')                    #for each line, find td tag with class as data
        genre = hyperlink.get_text()                                    #get text from data class 
        genre_clean = genre.replace(',','')
        genres = genre_clean.split()
    return(genres)                                                      #process the text and return genres

In [67]:
class Movie:                                                            #Create a class as Movie
    def __init__(self, name, cast, genre):                              #initialize the class with parameters as name, cast and genre
        self.name = name
        self.cast = cast
        self.genre = genre
        
class All_Movies:                                                       #create a class as All_Movies
    def __init__(self, name):                                           #initialize the class with parameter name
        self.name = name
        self.Movies = []                                                #create an empty list
        
    def add_movie(self, Movie):                                         #Define a function add_movie to append the movie in the list Movies
        self.Movies.append(Movie)  

In [68]:
all_movies = All_Movies("500 movies")
import time                                                             #Import time module to handle time related tasks
n = 0                                                                   #Initialize n as 0
for movie in container:                                                 #looping through all 500 movies in the container            
    n = n+1
    print(n)
    movie_name = get_name()                                             #get the movie name using get_name() function    
    print(movie_name)                                                   #print the movie name
    movie_link_tag = movie.find('a', class_ = 'title')                  #Steps to get the detail link to find the cast and genre
    movie_link = movie_link_tag.get('href', None)
    movie_link = 'https://www.metacritic.com'+ movie_link
    detail_page = requests.get(movie_link, headers = headers)
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")
    div = detail_soup('div', class_= 'see_all pad_btm1')
    for line in div:
        hyperlink = line.find('a')
        detail_link = hyperlink.get('href', None) 
        detail_link = 'https://www.metacritic.com'+ detail_link        #Once we have the detailed links,
        print(detail_link)                                             #Print the detailed link
        pri_cast = get_pri_cast(detail_link)                           #Call the get_pri_cast() method to get the principle cast details
        cast = get_cast(detail_link)                                   #Call the get_cast() method to get the cast details
        comb_cast = []                                                 #Create an empty list to combine principle cast and cast
        empty = ['Not found']                                          #Create a new list having only one value as Not Found
        if cast == None and pri_cast == None:                          #If cast and pri_cast does not have any value,
            comb_cast.extend(empty)                                    #set comb_cast as empty
        elif cast == None:                                             #If cast is None
            comb_cast.extend(pri_cast)                                 #extend pri_cast to comb_cast
        elif pri_cast == None:                                         #If pri_cast is None
            comb_cast.extend(cast)                                     #extend cast to comb_cast
        else:
            comb_cast.extend(pri_cast)                                 #else, exten both cast and pri_cast to comb_cast
            comb_cast.extend(cast)
        print(comb_cast)                                               #Print the combined cast
        genre = get_genre_new(detail_link)                             #get the genre name using get_genre_new() function
        print(genre)                                                   #print the genre
    movie_info = [movie_name, comb_cast, genre]                        #create a list movie_info as movie_name, comb_cast and genre
    lst = Movie(movie_name,  comb_cast, genre)                         #call the function Moviw
    all_movies.add_movie(lst)                                          #add all_movies


1
Citizen Kane
https://www.metacritic.com/movie/citizen-kane-1941/details
['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']
['Drama', 'Mystery']
2
The Godfather
https://www.metacritic.com/movie/the-godfather/details
['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']
['Drama', 'Thriller', 'Crime']
3
Rear Window
https://www.metacritic.com/movie/rear-window/details
['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'J

['Drama', 'Mystery', 'Crime']
22
All About Eve
https://www.metacritic.com/movie/all-about-eve/details
['Anne Baxter', 'Barbara Bates', 'Bette Davis', 'Celeste Holm', 'Gary Merrill', 'George Sanders', 'Gregory Ratoff', 'Hugh Marlowe', 'Marilyn Monroe', 'Thelma Ritter', 'Barbara White', 'Craig Hill', 'Leland Harris', 'Randy Stuart', 'Walter Hampden']
['Drama']
23
Jules and Jim
https://www.metacritic.com/movie/jules-and-jim/details
['Jeanne Moreau', 'Oskar Werner', 'Anny Nelsen', 'Bernard Largemain', 'Danielle Bassiak', 'Dominique Lacarrière', 'Elen Bober', 'Henri Serre', 'Kate Noelle', 'Marie Dubois', 'Michel Subor', 'Pierre Fabre', 'Sabine Haudepin', 'Serge Rezvani', 'Vanna Urbino']
['Drama', 'Romance']
24
The Wild Bunch
https://www.metacritic.com/movie/the-wild-bunch/details
['Ben Johnson', "Edmond O'Brien", 'Emilio Fernández', 'Ernest Borgnine', 'Jaime Sanchez', 'L.Q. Jones', 'Robert Ryan', 'Strother Martin', 'Warren Oates', 'William Holden', 'Albert Dekker', 'Bo Hopkins', 'Dub Taylor

['Biography', 'Drama', 'History']
44
Manchester by the Sea
https://www.metacritic.com/movie/manchester-by-the-sea/details
['Casey Affleck', 'Kyle Chandler', 'Lucas Hedges', 'Michelle Williams', 'Amanda Blattner', 'Anna Baryshnikov', 'Anthony Estrella', 'Ben Hanson', "Ben O'Brien", 'Brian Chamberlain', 'C.J. Wilson', 'Casey Affleck', 'Chaunty Spillane', 'Danae Nason', 'Erica McDermott', 'Gretchen Mol', 'Heather Burns', 'Jeffrey Corazzini', 'Josh Hamilton', 'Kara Hayward', 'Kyle Chandler', 'Lewis D. Wheeler', 'Liam McNeill', 'Lucas Hedges', 'Mark Burzenski', 'Mary Mallen', 'Matthew Broderick', 'Michelle Williams', 'Missy Yager', 'Oscar Wahlberg', 'Quincy Tyler Bernstine', 'Richard Donelly', 'Robert Sella', 'Ruibo Qian', 'Stephanie Ann Saunders', 'Stephen Henderson', 'Susan Pourfar', 'Tate Donovan', 'Tom Kemp', 'Virginia Loring Cooke']
['Drama']
45
Killer of Sheep
https://www.metacritic.com/movie/killer-of-sheep/details
['Angela Burnett', 'Charles Bracy', 'Delores Farley', 'Dorothy Stenge

https://www.metacritic.com/movie/portrait-of-a-lady-on-fire/details
['Adele Haenel', 'Armande Boulanger', 'Cécile Morel', 'Christel Baras', 'Clément Bouyssou', 'Guy Delamarche', 'Luàna Bajrami', 'Michèle Clément', 'Noémie Merlant', 'Valeria Golino']
['Drama', 'History', 'Romance']
65
I Am Not Your Negro
https://www.metacritic.com/movie/i-am-not-your-negro/details
['Dick Cavett', 'James Baldwin', 'Samuel L. Jackson', 'Shumerria Harris']
['Documentary']
66
Woodstock
https://www.metacritic.com/movie/woodstock-1970/details
['Arlo Guthrie', 'Country Joe and the Fish', 'Crosby Stills & Nash', 'Joan Baez', 'Joe Cocker', 'Richie Havens', 'Sha-Na-Na', 'Ten Years After', 'The Who', 'Bob Davis', 'Canned Heat', 'Jimi Hendrix', 'John Sebastian', 'Santana', 'Sly and the Family Stone']
['History', 'Music', 'Documentary']
67
Double Indemnity
https://www.metacritic.com/movie/double-indemnity/details
['Barbara Stanwyck', 'Byron Barr', 'Edward G. Robinson', 'Fortunio Bonanova', 'Fred MacMurray', 'Jean He

['Drama', 'Thriller', 'Crime']
83
The Servant
https://www.metacritic.com/movie/the-servant/details
['Dirk Bogarde', 'James Fox', 'Sarah Miles', 'Wendy Craig', 'Alun Owen', 'Ann Firbank', 'Brian Phelan', 'Catherine Lacey', 'Derek Tansley', 'Doris Knox', 'Harold Pinter', 'Hazel Terry', 'Jill Melford', 'Patrick Magee', 'Richard Vernon']
['Drama']
84
Pulp Fiction
https://www.metacritic.com/movie/pulp-fiction/details
['Bruce Willis', 'John Travolta', 'Samuel L. Jackson', 'Amanda Plummer', 'Duane Whitaker', 'Eric Stoltz', 'Frank Whaley', 'Maria de Medeiros', 'Paul Calderon', 'Peter Greene', 'Phil LaMarr', 'Rosanna Arquette', 'Tim Roth', 'Uma Thurman', 'Ving Rhames']
['Drama', 'Thriller', 'Crime']
85
The Manchurian Candidate
https://www.metacritic.com/movie/the-manchurian-candidate-1962/details
['Frank Sinatra', 'Laurence Harvey', 'Albert Paulsen', 'Angela Lansbury', 'Barry Kelley', 'Douglas Henderson', 'Henry Silva', 'James Edwards', 'James Gregory', 'Janet Leigh', 'John McGiver', 'Khigh Dhi

...

https://www.metacritic.com/movie/nobody-knows/details
['Ayu Kitaura', 'Hiei Kimura', 'Momoko Shimizu', 'Yûya Yagira', 'Hanae Kan', 'Kazuyoshi Kushida', "Ken'ichi Endô", 'Ryo Kase', 'Sei Hiraizumi', 'Susumu Terajima', 'Takako Tate', 'You', 'Yûichi Kimura', 'Yukiko Okamoto']
['Drama']
410
Life Is Sweet
https://www.metacritic.com/movie/life-is-sweet/details
['Alison Steadman', 'Jim Broadbent', 'Claire Skinner', 'David Neilson', 'David Thewlis', 'Harriet Thorpe', 'Jack Thorpe-Baker', 'Jane Horrocks', 'Moya Brady', 'Paul Trussel', 'Stephen Rea', 'Timothy Spall']
['Drama', 'Comedy']
411
Faces
https://www.metacritic.com/movie/faces/details
['Darlene Conley', 'Dorothy Gulliver', 'Fred Draper', 'Gena Rowlands', 'Gene Darfler', 'Joanne Moore Jordan', 'John Marley', 'Lynn Carlin', 'Seymour Cassel', 'Val Avery', 'Anita White', 'Ann Shirley', 'Dave Mazzie', 'Elizabeth Deering', 'Julie Gambol']
['Drama']
412
Mary Poppins
https://www.metacritic.com/movie/mary-poppins/details
['Dick Van Dyke', 'Julie 

https://www.metacritic.com/movie/imitation-of-life/details
["Dan O'Herlihy", 'John Gavin', 'John Vivyan', 'Juanita Moore', 'Karin Dicker', 'Lana Turner', 'Robert Alda', 'Sandra Dee', 'Susan Kohner', 'Terry Burnham', 'Ann Robinson', 'David Tomack', 'Lee Goodman', 'Sandra Gould', 'Troy Donahue']
['Drama']
434
Pain and Glory
https://www.metacritic.com/movie/pain-and-glory/details
['Agustín Almodóvar', 'Antonio Banderas', 'Asier Etxeandia', 'Asier Flores', 'Cecilia Roth', 'César Vicente', 'Eva Martín', 'Julián López', 'Julieta Serrano', 'Leonardo Sbaraglia', 'Nora Navas', 'Penélope Cruz', 'Raúl Arévalo', 'Rosalía', 'Susi Sánchez', 'Agustín Almodóvar', 'Alba García', 'César Vicente', 'Eva Martín', 'Julián López', 'Pedro Casablanc', 'Raúl Arévalo', 'Sara Sierra', 'Susi Sánchez']
['Drama']
435
Genèse (Genesis)
https://www.metacritic.com/movie/gencse-genesis/details
['Antoine Marchand Gagnon', "Ariane L'Heureux", 'Brett Dier', 'Émile Bilodeau', 'Emilie Bierre', 'Étienne Galloy', 'Marc Beaupré'

['Drama']
457
Spartacus
https://www.metacritic.com/movie/spartacus/details
['Kirk Douglas', 'Laurence Olivier', 'Charles Laughton', 'Charles McGraw', 'Harold J. Stone', 'Herbert Lom', 'Jean Simmons', 'Joanna Barnes', 'John Dall', 'John Gavin', 'John Ireland', 'Nina Foch', 'Peter Brocco', 'Peter Ustinov', 'Woody Strode']
['Adventure', 'Biography', 'Drama', 'History', 'War']
458
Small Axe: Education
https://www.metacritic.com/movie/small-axe-education/details
['Calum Callaghan', 'Ciarán Owens', 'Jim Sturgeon', 'John Boyega', 'Joy Richardson', 'Marina Hayter', 'Martin Portlock', 'Stephen Boxer', 'Steve Toussaint', 'Tyrone Huntley', 'Conor Lowson', 'Eddie-Joe Robinson', 'Jaden Oshenye', 'Nathan Vidal', 'Robert Godley-Harrod']
['Drama']
459
About Endlessness
https://www.metacritic.com/movie/about-endlessness/details
['Amanda Davies', 'Anders Hellström', 'Ania Nova', 'Anja Broms', 'Anton Forsdik', 'Bengt Bergius', 'Bertil J. Nyberg', 'Conny Block', 'Florencio Urbano', 'Gloria Ormandlaky', 'I

['Drama', 'Romance', 'Western']
478
Cemetery of Splendor
https://www.metacritic.com/movie/cemetery-of-splendor/details
['Apinya Unphanlam', 'Banlop Lomnoi', 'Bhattaratorn Senkraigul', 'Boonyarak Bodlakorn', 'Jarinpattra Rueangram', 'Jenjira Jansuda', 'Jenjira Pongpas', 'Kammanit Sansuklerd', 'Petcharat Chaiburi', 'Pongsadhorn Lertsukon', 'Richard Abramson', 'Sakda Kaewbuadee', 'Sasipim Piwansenee', 'Sujittraporn Wongsrikeaw', 'Tawatchai Buawat', 'Wacharee Nagvichien']
['Drama']
479
The Kid with a Bike
https://www.metacritic.com/movie/the-kid-with-a-bike/details
['Cécile De France', 'Egon Di Mateo', 'Fabrizio Rongione', 'Jérémie Renier', 'Olivier Gourmet', 'Thomas Doret', 'Batiste Sornin', 'Carl Jadot', 'Cécile De France', 'Claudy Delfosse', 'Egon Di Mateo', 'Fabrizio Rongione', 'Frédéric Dussenne', 'Hicham Slaoui', 'Jean-Michel Balthazar', 'Jérémie Renier', 'Myriem Akeddiou', 'Olivier Gourmet', 'Samuel De Rijk', 'Sandra Raco', 'Thomas Doret']
['Drama']
480
Mutiny on the Bounty
https://

https://www.metacritic.com/movie/the-english-patient/details
['Juliette Binoche', 'Kristin Scott Thomas', 'Ralph Fiennes', 'Willem Dafoe', 'Clive Merrison', 'Colin Firth', 'Geordie Johnson', 'Hichem Rostom', 'Julian Wadham', 'Jürgen Prochnow', 'Kevin Whately', 'Naveen Andrews', 'Nino Castelnuovo', 'Peter Rühring', 'Torri Higginson']
['Drama', 'Romance', 'War']


In [69]:
for i in all_movies.Movies:                         #For all the movies in all_movies.Movies                      
    print(i.name, i.cast, i.genre)                  #print name,cast and genre of 500 movies
print(i)

Citizen Kane ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland'] ['Drama', 'Mystery']
The Godfather ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire'] ['Drama', 'Thriller', 'Crime']
Rear Window ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper'] ['Mystery', 'Thriller']
Casablanca ['Humphrey Bogart', 'Ingrid Bergman', 'Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wils

# Import the content into CSV

In [10]:
import csv                                                                 #Import csv library to import content from python to CSV

with open('yash_class.csv', mode='w', newline='') as movie_file:           #Open a file called yash_class,csv in write mode as a movie_file
    movie_writer = csv.writer(movie_file)                                  
    for i in all_movies.Movies:                                            #for all data in all_movies.Movies,
        movie_writer.writerow([i.name, i.cast, i.genre])                   #Write rows as movie_name, cast and genre of the movie into excel

# Perform task 1,2 and 3 by selecting movie, people or comparison

In [70]:
while True:                                                    #condition to check if a proper option is selected
    inp = input('What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’)')
    initial_inp = inp.lower()                                  #convert the input to lower case to handle exceptions
    if initial_inp in ('movie','people','comparison'):         #if the input is from movie, people or comparison do the required task
        break
    else:                                                      #else, ask for a new choice
        print('enter a valid choice from movie, people or comparison')

if initial_inp == 'movie':                                     #if the input is movie
    inp2 = input('What movie do you want to check?')           #ask user to enter the movie name
    second_inp = inp2.title()                                  #convert the input to title with first letter as capital for all words
    for i in all_movies.Movies:                                #loop through all movies to find if input == movie_name
        if i.name == second_inp:                               #if the movie is found, print the cast and genre of the movie
            print('The cast of movie',i.name,'includes :',','.join(i.cast))
            print('The Genre of movie',i.name,'is :',','.join(i.genre))
if initial_inp == 'people':                                    #if the input is people
    inp2 = input('What do you want to check?')                 #Ask user to enter the person's name
    second_inp = inp2.title()                                  #convert the input to title with first letter as capital for all words
    movies_acted = []                                          #create an empty list as movies_acted
    genre_acted = [] 
    for i in all_movies.Movies:                                #for all movies,
        for j in range(len(i.cast)):                           #take all the items in cast list of that movie
            if i.cast[j] == second_inp:                        #if that item is equal to the user's input,
                movies_acted.append(i.name)                    #append the movie name in movies_acted
                genre_acted.extend(i.genre)                    #append the movie name in genre_acted
    print(second_inp,'has acted in ',','.join(movies_acted))   #print the movies the actor has acted in
    genre_count = dict()                                       #create an empty dictionary
    for i in genre_acted:                                      #for all genre in genre_acted
        genre_count[i] = genre_count.get(i, 0) + 1             #count the genres and print a consolidated dictionary
    print('His most often played genres are :' + str(genre_count))
    
if initial_inp == 'comparison':                                #if the input is comparison
    inp3 = input("Enter the first person's name to compare ?") #Ask user to enter the first person's name
    third_inp = inp3.title()                                   #convert the input to title with first letter as capital for all words
    inp4 = input("Enter the second person's name to compare ?")#Ask user to enter the first person's name
    fourth_inp = inp4.title()                                  #convert the input to title with first letter as capital for all words
    movies1_acted = []                                         #create an empty list as movies1_acted
    movies2_acted = []                                         #create an empty list as movies2_acted
    genre1_acted = []                                          #create an empty list as genres1_acted
    genre2_acted = []                                          #create an empty list as genres2_acted
    for i in all_movies.Movies:                                #for all movies,
        for j in range(len(i.cast)):                           #take all the items in cast list of that movie
            if i.cast[j] == third_inp:                         #if that item is equal to the user's input,
                movies1_acted.append(i.name)                   #append the movie name in movies1_acted
                genre1_acted.extend(i.genre)                   #extend the genre name in genre1_acted
    print(third_inp,'has acted in ',','.join(movies1_acted))   #print the movies the actor has acted in
    genre1_count = dict()                                      #create an empty dictionary
    for i in genre1_acted:                                     #for all genre in genre1_acted 
        genre1_count[i] = genre1_count.get(i, 0) + 1           #count the genres and print a consolidated dictionary
    print('His most often played genres are :' + str(genre1_count))
          
    for i in all_movies.Movies:                                #for all movies,
        for j in range(len(i.cast)):                           #take all the items in cast list of that movie 
            if i.cast[j] == fourth_inp:                        #if that item is equal to the user's input,
                movies2_acted.append(i.name)                   #append the movie name in movies1_acted
                genre2_acted.extend(i.genre)                   #extend the genre name in genre1_acted 
    print(fourth_inp,'has acted in',','.join(movies2_acted))   #print the movies the actor has acted in
    genre2_count = dict()                                      #create an empty dictionary 
    for i in genre2_acted:                                     #for all genre in genre2_acted
        genre2_count[i] = genre2_count.get(i, 0) + 1           #count the genres and print a consolidated dictionary
    print('His most often played genres are :' + str(genre2_count))  

What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’)film
enter a valid choice from movie, people or comparison
What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’)movie
What movie do you want to check?Citizen Kane
The cast of movie Citizen Kane includes : Joseph Cotten,Orson Welles,Agnes Moorehead,Dorothy Comingore,Erskine Sanford,Everett Sloane,Fortunio Bonanova,George Coulouris,Georgia Backus,Gus Schilling,Harry Shannon,Paul Stewart,Philip Van Zandt,Ray Collins,Ruth Warrick,William Alland
The Genre of movie Citizen Kane is : Drama,Mystery


In [71]:
    set_actor1 = set(genre1_acted)                           #Create a set of genre1_acted
    set_actor2 = set(genre2_acted)                           #Create a set of genre2_acted
    combine_set = set_actor1.union(set_actor2)               #find the common genres by taking out a union of both sets
    print(combine_set)                                       #print the combined genre set

{'Drama', 'War', 'Fantasy', 'Family', 'Animation', 'Music', 'Action', 'Sci-Fi', 'Adventure', 'Comedy'}


In [72]:
actor1_dict = dict()                                         #Create an empty dictionary
for i in combine_set:                                        #for item in combine_set  
    actor1_dict[i] = 0                                       #set the genre as 0
print(actor1_dict)

actor2_dict = dict()                                         #Create an empty dictionary
for i in combine_set:                                        #for item in combine_set 
    actor2_dict[i] = 0                                       #set the genre as 0 
print(actor2_dict)

{'Drama': 0, 'War': 0, 'Fantasy': 0, 'Family': 0, 'Animation': 0, 'Music': 0, 'Action': 0, 'Sci-Fi': 0, 'Adventure': 0, 'Comedy': 0}
{'Drama': 0, 'War': 0, 'Fantasy': 0, 'Family': 0, 'Animation': 0, 'Music': 0, 'Action': 0, 'Sci-Fi': 0, 'Adventure': 0, 'Comedy': 0}


In [73]:
for i in genre1_acted:                                       #for item in genre1_acted 
    actor1_dict[i] = actor1_dict[i] + 1                      #count the total number of a particular genre
print(actor1_dict)


for i in genre2_acted:                                       #for item in genre2_acted 
    actor2_dict[i] = actor2_dict[i] + 1                      #count the total number of a particular genre
print(actor2_dict)

{'Drama': 1, 'War': 1, 'Fantasy': 3, 'Family': 3, 'Animation': 3, 'Music': 0, 'Action': 1, 'Sci-Fi': 0, 'Adventure': 3, 'Comedy': 3}
{'Drama': 3, 'War': 0, 'Fantasy': 0, 'Family': 0, 'Animation': 0, 'Music': 1, 'Action': 2, 'Sci-Fi': 2, 'Adventure': 2, 'Comedy': 0}


# Create a vector indicating values for both the actor genres

In [74]:

actor1_vector = list(actor1_dict.values())
actor2_vector = list(actor2_dict.values())
print(actor1_vector)
print(actor2_vector)

[1, 1, 3, 3, 3, 0, 1, 0, 3, 3]
[3, 0, 0, 0, 0, 1, 2, 2, 2, 0]


# Calculate the cosine similarity

In [75]:

divident = 0 
for i in range(len(actor1_vector)):
    summing = actor1_vector[i] * actor2_vector[i]
    divident = divident + summing
a = 0
b = 0
for i in range(len(actor1_vector)):
    sum1 = (actor1_vector[i])**2
    a = a + sum1
    sum2 = (actor2_vector[i])**2
    b = b + sum2
divisor = (a**0.5)*(b**0.5)
cosine_similarity = divident/divisor
print("Based on that, they have a cosine similarity score of",cosine_similarity)

Based on that, they have a cosine similarity score of 0.338501600193165


# Perform the writing task to prepare a similarity report

In [76]:
cast1='Agnes Moorehead'                               #Initialize the first input 
cast2='Stephen Stanton'                               #Initialize the second input                   
cast3='Marlene Siskel'                                #Initialize the third input   
cast4='Gene Siskel'                                   #Initialize the fourth input  
cast5='Chadwick Boseman'                              #Initialize the fifth input 

genre_c1 = []                                         #create an empty list
genre_c2 = []
genre_c3 = []
genre_c4 = []
genre_c5 = []

for i in all_movies.Movies:                           #Print the genres of the movies acted by an actor1
        for j in range(len(i.cast)):
            if i.cast[j] == cast1:
                genre_c1.extend(i.genre)
#print(genre_c1)
genre_count = dict()
for i in genre_c1:
    genre_count[i] = genre_count.get(i, 0) + 1
print(cast1,'acted in movies of genres :' + str(genre_count))
print('--------------------------------------------------------------------------------')
for i in all_movies.Movies:                           #Print the genres of the movies acted by an actor2 
        for j in range(len(i.cast)):
            if i.cast[j] == cast2:
                genre_c2.extend(i.genre)
#print(genre_c2)
genre_count = dict()
for i in genre_c2:
    genre_count[i] = genre_count.get(i, 0) + 1
print(cast2,'acted in movies of genres  :' + str(genre_count))
print('--------------------------------------------------------------------------------')
for i in all_movies.Movies:                          #Print the genres of the movies acted by an actor3
        for j in range(len(i.cast)):
            if i.cast[j] == cast3:
                genre_c3.extend(i.genre)
#print(genre_c3)
genre_count = dict()
for i in genre_c3:
    genre_count[i] = genre_count.get(i, 0) + 1
print(cast3,'acted in movies of genres  :' + str(genre_count))
print('--------------------------------------------------------------------------------')
for i in all_movies.Movies:                         #Print the genres of the movies acted by an actor4
        for j in range(len(i.cast)):
            if i.cast[j] == cast4:
                genre_c4.extend(i.genre)
#print(genre_c4)
genre_count = dict()
for i in genre_c4:
    genre_count[i] = genre_count.get(i, 0) + 1
print(cast4,'acted in movies of genres  :' + str(genre_count))
print('--------------------------------------------------------------------------------')
for i in all_movies.Movies:                         #Print the genres of the movies acted by an actor5
        for j in range(len(i.cast)):
            if i.cast[j] == cast5:
                genre_c5.extend(i.genre)
#print(genre_c5)
genre_count = dict()
for i in genre_c5:
    genre_count[i] = genre_count.get(i, 0) + 1
print(cast5,'acted in movies of genres  :' + str(genre_count))
print("-------------------------------------------------------------------------------------------")
print('The combined cast for five actors is given below')
all_genre_five=list()                              #Print the consolidated list containing genres from all actors with count of their occurence 
all_genre_five.extend(genre_c1)
all_genre_five.extend(genre_c2)
all_genre_five.extend(genre_c3)
all_genre_five.extend(genre_c4)
all_genre_five.extend(genre_c5)
#print(all_genre_five,len(all_genre_five))
genre_count = dict()          
for i in all_genre_five:
    genre_count[i] = genre_count.get(i, 0) + 1
print('overall genre of 5 actors includes :' + str(genre_count))

Agnes Moorehead acted in movies of genres :{'Drama': 2, 'Mystery': 1, 'Romance': 1}
--------------------------------------------------------------------------------
Stephen Stanton acted in movies of genres  :{'Biography': 1, 'Documentary': 1}
--------------------------------------------------------------------------------
Marlene Siskel acted in movies of genres  :{'Biography': 1, 'Documentary': 1}
--------------------------------------------------------------------------------
Gene Siskel acted in movies of genres  :{'Biography': 2, 'Documentary': 2}
--------------------------------------------------------------------------------
Chadwick Boseman acted in movies of genres  :{'Action': 2, 'Adventure': 2, 'Sci-Fi': 2, 'Drama': 3, 'Music': 1}
-------------------------------------------------------------------------------------------
The combined cast for five actors is given below
overall genre of 5 actors includes :{'Drama': 5, 'Mystery': 1, 'Romance': 1, 'Biography': 4, 'Documentary':